<a href="https://colab.research.google.com/github/Gabrielkwia/Automacao-de-Cobrancas-por-Email/blob/main/AnaliseDeDados2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando bibliotecas para o projeto (Pandas para análise de dados & Datime para trabalhar com datas e horas)
import pandas as pd
from datetime import datetime
# Para envio de email
import smtplib
from email.mime.text import MIMEText

In [ ]:
# Definindo o caminho para o arquivo e definine uma variavel para ele "df"
df = pd.read_excel("/content/drive/MyDrive/AnaliseDeDados/contas a receber.xlsx")

In [ ]:
# Mostra as informações do seu DataFrame
df.info()

In [ ]:
# Mostra a coluna "Valor em aberto"
df["Valor em aberto"]

In [ ]:
# O unique mostra todos os valores na coluna sem repetir eles
df["Valor em aberto"].unique()

In [ ]:
# Replace substitui todos os pontos por nada, ou seja, remove os pontos
df["Valor em aberto"] = df["Valor em aberto"].str.replace(".", "")
df["Valor em aberto"].unique()

In [ ]:
# Replace troca a virgula por pontos e remove o R$
df["Valor em aberto"] = df["Valor em aberto"].str.replace(",", ".").str.replace("R$ ", "")
df["Valor em aberto"].unique()

In [ ]:
# df["Valor em aberto"] = df["Valor em aberto"].str.replace("R$ ", "")
# df["Valor em aberto"].unique()

In [ ]:
# Transformando em numerico
df["Valor em aberto"] = pd.to_numeric(df["Valor em aberto"])

In [ ]:
# sort_values ordena os valores em forma crescente
df.sort_values(by = "Valor em aberto")

In [ ]:
# Cria uma nova variavel e armazena os dados filtrados que estejam cm a coluna "Status" como "Atrasada"
df_atrasados = df[ df["Status"] == "Atrasado" ]

In [ ]:
# Mostra todos os que estão com status atrasado
df_atrasados

In [ ]:
# Mostra o numero de devedores usando o f-string
print(f"O numero de devedores do periodo é de {df_atrasados['CPF'].nunique()} clientes")

In [ ]:
# Esse loop passa por cada linha do Dataframe e imprime os dados correspondentes
for x in df_atrasados.iterrows():
  print(x)

In [ ]:
# Configurações Iniciais para disparar o email / Conectando com o servidor
smtp_server = "smtp.gmail.com"
smtp_port = 587
email_usuario = "gabrielmaneira048@gmail.com"
email_senha = "fkzi wzez vjur vxlf"
remetente_nome = "Financeiro"

print("Conectando ao servidor")
server = smtplib.SMTP(smtp_server, smtp_port)
server.starttls()
server.login(email_usuario, email_senha)
print("Login realizado com sucesso")

# Organiza as informações de cada conta atrasada para que possam ser utilizadas, para gerar um aviso de e-mail personalizado
for indice, linha in df_atrasados.iterrows():
  destinatario = linha['E-mail']
  nf = linha["NF"]
  valor = linha["Valor em aberto"]
  prazo = linha["Data Prevista para pagamento"]
  assunto = f"Aviso sobre atraso no pagamento da {nf}"

# Mensagem que sera enviada no email de todos que estao com conta em atraso
  corpo_mensagem = f'''

  Prezado(a) cliente,

  Notamos que não houve o pagamento referente ao seu pedido com nota {nf}, no valor de {valor} que teve o prazo
  limite na data de {prazo}. Qual a expectativa para quitar o debito em aberto?

  Atenciosamente,
  {remetente_nome}
  '''
# Constroi o corpo do email, define o tipo de texto, assunto, remetende e distinatário
  mensagem = MIMEText(corpo_mensagem, 'plain', 'utf-8')
  mensagem["Subject"] = assunto
  mensagem["From"] = f"{remetente_nome} <{email_usuario}>"
  mensagem["To"] = destinatario

# Envia de fato o email ao destinatário
  server.sendmail(email_usuario, destinatario, mensagem.as_string())

  print(f" Email enviado com sucesso referente a nota {nf}")

